In [1]:
import collections.abc
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

import numpy as np
import scipy.stats as st
import pandas as pd
import os
import scipy
import warnings
import scikit_posthocs as sp

import iqplot
import bebi103

import bokeh.io
import bokeh.plotting
import bokeh.layouts
bokeh.io.output_notebook()

/Users/anastasiaoguienko/miniconda3/envs/bebi103/lib/python3.13/site-packages/bebi103/viz.py:38: UserWarning: DataShader import failed with error "cannot import name 'Callable' from 'collections' (/Users/anastasiaoguienko/miniconda3/envs/bebi103/lib/python3.13/collections/__init__.py)".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(


Loading BokehJS ...

### Specifying the data, models, main effects

In [2]:
filename = 'MotorBehaviordata_NAC_Livia022924.xlsx'
data_path = os.path.join("..", "..", "Data", "NAC", filename)
sheet_name = 'Pole and Beam'

# Group factors
effect_1 = 'Genotype' 
effect_2 = 'Treatment'

# Type pf test for current analysis
mtest = 'Pole_T'

# List of models to fit into and assess
# Encoded models: 'lognormal', 'lognormal_mixed', 'normal', 'normal_mixed', 
models = ['lognormal', 'lognormal_mixed', 'normal', 'normal_mixed']

# Whether or not the experiment has several cohorts and a dedicated cohort column
cohort_col = False 

### Plotting characteristics

Specifying palettes and plotting order depending on the data used (SPF/GF, NAC, Auronafin, DJKO experiments).

In [3]:
# Palettes:
palette_CI = ["#bdbdbd", "#000000", "#bdbdbd", "#000000"]

# Plotting orders:
order = {'WT_V': 1, 'ASO_V': 2, 'WT_NAC': 3, 'ASO_NAC': 4}

### Exploratory data analysis

Tidying up the data.

In [4]:
pole_beam_df = pd.read_excel(data_path, sheet_name=sheet_name)
pole_beam_df = pole_beam_df.rename(columns={'ID ':'ID', 'Group ':'Group', 'treatment':'Treatment'})

if (effect_1 not in pole_beam_df.columns) or (effect_2 not in pole_beam_df.columns):
    pole_beam_df[[effect_1, effect_2]] = pole_beam_df['Group'].str.split('-', expand = True)

pole_beam_df

,Datetest,ID,Group,Genotype,Treatment,Beam_T1,Beam_T2,Beam_T3,Pole_T1,Pole_T2,Pole_T3
0,111221,3556,ASO-V,ASO,V,15.47,15.13,15.37,37.00,23.00,60.00
1,111221,3560,ASO-V,ASO,V,9.09,19.94,28.88,10.87,60.00,60.00
2,111221,3574,ASO-V,ASO,V,60.00,60.00,60.00,7.37,3.04,60.00
3,111221,3577,ASO-V,ASO,V,60.00,60.00,60.00,9.10,4.82,60.00
4,111221,3599,ASO-V,ASO,V,7.67,5.83,19.00,60.00,60.00,60.00
5,111221,3602,ASO-V,ASO,V,20.00,14.53,60.00,14.02,6.14,60.00
6,111221,334,ASO-V,ASO,V,60.00,60.00,60.00,60.00,60.00,14.78
7,111221,335,ASO-V,ASO,V,26.34,60.00,9.18,9.53,6.12,16.80
8,111221,3558,WT-V,WT,V,2.19,2.18,5.56,8.00,10.04,14.57
9,111221,3559,WT-V,WT,V,2.72,6.74,3.45,4.82,4.81,4.03


Creating the working data frame only with the values of interest, removing missing values and cases where mice jumped off of the pole/beam.

In [5]:
pole_beam_df = pole_beam_df.replace(to_replace = 'jumped', value = float("NaN"))

res = [i for i in list(pole_beam_df.columns) if (mtest) in i]

if cohort_col:
    col_list = [cohort_col, effect_1, effect_2, 'ID']
else:
    col_list = [effect_1, effect_2, 'ID']

pole_df = pole_beam_df[col_list + res].copy()
pole_df_long = pole_df.melt(id_vars = col_list, var_name='Trial', value_name='Time')
pole_df_long['Trial'] = pole_df_long['Trial'].str[-2:]
pole_df_long['ID'] = pole_df_long['ID'].astype('str')
pole_df_long = pole_df_long.dropna()
pole_df_long = pole_df_long.groupby(['Genotype', 'Treatment', 'ID'])['Time'].median().reset_index()

Creating dictionaries with groups as keys and experimentally measured values.

In [6]:
group_vals = {}

effect1_lst = pole_df_long[effect_1].unique()
effect2_lst = pole_df_long[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = pole_df_long.loc[(pole_df_long[effect_1] == i) & (pole_df_long[effect_2] == j), 'Time'].values
        if len(n) != 0:
            group_vals[name] = n

### Defining all the necessary functions for the MLE calculations

In [7]:
def log_like_iid_lognormal_mixed(params, n):
    """Log likelihood for i.i.d. lognormal measurements mixed with Kronecker delta function.

    Parameters
    ----------
    params : array
        Parameters mu, sigma, omega.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma, omega = params

    if mu <= 0 or sigma <= 0 or omega <=0 or omega >= 1:
        return -np.inf

    target = 0
    
    for i in n:
        if i == 60:
            target += scipy.special.logsumexp([np.log(1 - omega), np.log(omega) + np.log(1 - st.lognorm.cdf(i, sigma, scale=np.exp(mu)))])
        else:
            target += np.log(omega) + st.lognorm.logpdf(i, sigma, scale=np.exp(mu))
                                              

    return target

In [8]:
def log_like_iid_lognormal(params, n):
    """Log likelihood for i.i.d. lognormal measurements.

    Parameters
    ----------
    params : array
        Parameters mu, sigma.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma = params

    if mu <= 0 or sigma <= 0:
        return -np.inf


    target = 0

    cdf_val = np.log(1 - st.lognorm.cdf(60, sigma, scale=np.exp(mu)))
    
    for i in n:
        if i == 60:
            target += cdf_val
        else:
            target += st.lognorm.logpdf(i, sigma, scale=np.exp(mu))

    return target
    

In [9]:
def log_like_iid_normal_mixed(params, n):
    """Log likelihood for i.i.d. normal measurements mixed with Kronecker delta function.

    Parameters
    ----------
    params : array
        Parameters mu, sigma, omega.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma, omega = params

    if mu <= 0 or sigma <= 0 or omega <=0 or omega >= 1:
        return -np.inf

    target = 0

    cdf_val = np.log(1 - st.norm.cdf(60, mu, sigma))
    
    for i in n:
        if i == 60:
            target += scipy.special.logsumexp([np.log(1 - omega), np.log(omega) + cdf_val])
        else:
            target += np.log(omega) + st.norm.logpdf(i, mu, sigma)
                                              

    return target

In [10]:
def log_like_iid_normal(params, n):
    """Log likelihood for i.i.d. normal measurements.

    Parameters
    ----------
    params : array
        Parameters mu, sigma.
    n : array
        Array of data points.

    Returns
    -------
    output : float
        Log-likelihood.
    """
    mu, sigma = params

    if mu <= 0 or sigma <= 0:
        return -np.inf


    target = 0

    cdf_val = np.log(1 - st.norm.cdf(60, mu, sigma))
    
    for i in n:
        if i == 60:
            target += cdf_val
        else:
            target += st.norm.logpdf(i, mu, sigma)

    return target
    

In [11]:
def gen_lognormal_mixed(params, size, rng):
    """Draws a sample out of the mixed lognormal distribution with Kronecker delta function
    parametrized by mu, sigma and omega.

    Parameters
    ----------
    params : array
        Parameters mu, sigma, omega.
    size : int
        Number of data poinst in a sample.
    rng: Generator
        The initialized generator object.

    Returns
    -------
    output : array
        Newly generated sample.
    
    """
    mu, sigma, omega = params

    num_max = rng.binomial(size, (1 - omega))
    y_max = np.ones(num_max) * 60
    num_weib = size - num_max

    y_lognorm = st.lognorm.rvs(sigma, scale=np.exp(mu), size=num_weib)

    if len(y_max) == 0:
        y = y_lognorm
    else:
        y = np.concatenate((y_lognorm, y_max))

    y[y > 60] = 60
    
    return y

In [12]:
def log_like(params, n, model):
    """
    Log likelihood for i.i.d. measurements for the given model.
    
    Parameters
    ----------
    params : array
        Parameters mu, sigma.
    n : array
        Array of data points.
    model : string
        One of the following: 'lognormal', 'lognormal_mixed', 'normal', 'normal_mixed'

    Returns
    -------
    output : float
        Log-likelihood.
    """

    if model == 'lognormal':
        return log_like_iid_lognormal(params, n)
    elif model == 'lognormal_mixed':
        return log_like_iid_lognormal_mixed(params, n)
    elif model == 'normal':
        return log_like_iid_normal(params, n)
    elif model == 'normal_mixed':
        return log_like_iid_normal_mixed(params, n)
    else:
        raise ValueError('Pick an appropriate model!')
                                     

In [13]:
def mle_iid(n, model):
    """Performs maximum likelihood estimates for parameters for i.i.d.
    measurements of a chosen model;

    Parameters
    ----------
    n : array
        Array of data points.
    model : string
        One of the following: 'lognormal', 'lognormal_mixed', 'normal', 'normal_mixed'

    Returns
    -------
    output : float
        MLE for given model parameters.
    """

    if model == 'lognormal':
        init_guess = np.array([2, 15])
    elif model == 'lognormal_mixed':
        init_guess = np.array([2, 15, 0.5])
    elif model == 'normal':
        init_guess = np.array([10, 15])
    elif model == 'normal_mixed':
        init_guess = np.array([10, 15, 0.5])
    else:
        raise ValueError('Pick an appropriate model!')

    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        res = scipy.optimize.minimize(
            fun=lambda params, n: -log_like(params, n, model),
            x0=init_guess,
            args=(n,),
            method='Powell'
        )

        if res.success:
            return res.x
        else:
            raise RuntimeError('Convergence failed with message', res.message)

In [14]:
def draw_perm_sample(x, y):
    """Generate a permutation sample."""
    concat_data = np.concatenate((x, y))
    np.random.shuffle(concat_data)

    return concat_data[:len(x)], concat_data[len(x):]


def draw_perm_reps(x, y, stat_fun, size=1):
    """Generate array of permuation replicates."""
    return np.array([stat_fun(*draw_perm_sample(x, y)) for _ in range(size)])

def draw_perm_reps_diff_mean(x, y, size=1):
    """Generate array of permuation replicates."""
    out = np.empty(size)
    for i in range(size):
        x_perm, y_perm = draw_perm_sample(x, y)
        out[i] = np.abs(np.mean(x_perm) - np.mean(y_perm))

    return out

Plotting functions:

In [15]:
def plot_conf_int_pub(data, parameter_name, palette):
    """Creating a plot of confidence intervals of a given parameter for given data using the specified palette.
    The output is a plot with high resolution that can be exported as an .svg or .html file."""
    p = bebi103.viz.confints(
        data,
        title=parameter_name+' 95%CI, lognormal mixed model',
        palette=palette,
        frame_width=4000,
        frame_height=800,
        hidpi=True,
        line_width=5,
        marker_kwargs={"size":15}
    )
    
    p.title.text_font_size = '48pt'
    p.xaxis.major_label_text_font_size = "40pt"
    p.yaxis.major_label_text_font_size = "40pt"
    p.output_backend = 'svg'
    
    return p

def plot_conf_int_notebook(data, parameter_name, palette):
    """Creating a plot of confidence intervals of a given parameter for given data using the specified palette.
    The output is a plot that can be viewed in the notebook."""
    
    p = bebi103.viz.confints(
        data,
        title=parameter_name+' 95%CI, lognormal mixed model',
        palette=palette
    )
    
    return p

### Calculating MLEs and AICs for all the models

In [16]:
mles_dict = {}
models = ['lognormal', 'lognormal_mixed', 'normal', 'normal_mixed']

AICs = pd.DataFrame(index = models)

for model in models:
    mles_dict[model] = {}
    for group in group_vals.keys():
        if len(group_vals[group]) != 0:
            params = mle_iid(group_vals[group], model=model)
        
            mles_dict[model][group] = params
    
            _llk = log_like(params, group_vals[group], model = model)
            AICs.loc[model, group] = -2 * (_llk - len(params))

AICs

/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_1527/1038771715.py:24: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(60, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_1527/1038771715.py:24: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(60, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_1527/995081636.py:23: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(60, mu, sigma))
/var/folders/nh/70s95sxn08715hzmb75vtm7r0000gn/T/ipykernel_1527/995081636.py:23: RuntimeWarning: divide by zero encountered in log
  cdf_val = np.log(1 - st.norm.cdf(60, mu, sigma))


,ASO_NAC,ASO_V,WT_NAC,WT_V
lognormal,98.030752,80.224695,53.490706,47.407756
lognormal_mixed,99.535811,80.513167,55.490707,49.407757
normal,112.411239,89.458548,57.398956,48.524032
normal_mixed,111.949800,84.129832,59.398956,50.524033


## Mixed Lognormal model

Fitting data into the mixed log-normal model with Kronecker delta function.

\begin{align}
    &y' \sim \omega \text{ LogNormal}(\mu, \sigma) + (1 - \omega) \delta_{y60} \\
    &y = \begin{cases} 
      y' & \text{if } y' <60 \\ 
      60  & \text{if } y' \ge 60 \end{cases}
\end{align}

In [17]:
ln_mles = mles_dict['lognormal_mixed']
ln_mles

{'ASO_NAC': array([2.61277286, 0.73445493, 0.94500764]),
 'ASO_V': array([2.75742192, 0.7328836 , 0.69016629]),
 'WT_NAC': array([1.77476154, 0.48715897, 0.99999997]),
 'WT_V': array([1.86038742, 0.41988039, 0.99999997])}

In [18]:
bs_mle_samples_mx_ln = {}

Drawing bootstrap replicates of parameter MLEs for every group in the experiment.  
**Choose an appropriate amount for <code>n_jobs</code> according to the number of cores available on the machine. Windows users might need to set it to 1 to make it work. There is another option of using previously generated bootstrap samples - refer to the code below the following coding cell.**

In [80]:
for group in group_vals.keys():
    print(group)

    bs_mle_sample = bebi103.bootstrap.draw_bs_reps_mle(
        mle_iid,
        gen_lognormal_mixed,
        data=group_vals[group],
        mle_args=('lognormal_mixed',),
        size=10000,
        n_jobs=7
    )
    
    _df = pd.DataFrame(bs_mle_sample, columns=['mu', 'sigma', 'omega'])
    _df.to_csv(os.path.join("..", "Output", group+"nac_pole_bs_mle_samples.csv"))

    bs_mle_samples_mx_ln[group] = bs_mle_sample

ASO_NAC
ASO_V
WT_NAC
WT_V


**If you are unable to run the code cell above or it takes too long, you can use the code below.**

In [19]:
for group in group_vals.keys():
    _df = pd.read_csv(os.path.join("..", "Output", group+"nac_pole_bs_mle_samples.csv"), index_col=0)
    bs_mle_samples_mx_ln[group] = _df.to_numpy()

## Plotting:

### Graphical model assessment

Generating bootstrap samples for further graphical model assessment.

In [20]:
bs_samples_mx_ln = {}
rng = np.random.default_rng()

for group in sorted(group_vals, key=lambda x: order[x]):
    params = ln_mles[group]
    single_samples = np.array([gen_lognormal_mixed(params, size=len(group_vals[group]), rng=rng) for _ in range(100000)])
    bs_samples_mx_ln[group] = single_samples


Making Q-Q plots

In [21]:
qqplots_mx_ln =[]
precdf_plots_mx_ln = []

for group in sorted(group_vals, key=lambda x: order[x]):

    p = bebi103.viz.qqplot(
        data=group_vals[group],
        samples=bs_samples_mx_ln[group],
        x_axis_label="time of descent",
        y_axis_label="time of descent",
        title=group + ' Q-Q plot'       
    )
    qqplots_mx_ln.append(p)

    p1 = bebi103.viz.predictive_ecdf(
        samples=bs_samples_mx_ln[group], 
        data=group_vals[group], 
        x_axis_label="time",
        title=group + ' predictive ECDF'
    )
    precdf_plots_mx_ln.append(p1)

qq_lt_mx_ln = bokeh.layouts.row(qqplots_mx_ln)

bokeh.io.show(qq_lt_mx_ln)

Making predictive ECDF plots

In [22]:
pe_lt_mx_ln = bokeh.layouts.row(precdf_plots_mx_ln)

bokeh.io.show(pe_lt_mx_ln)

### Generating 95% confidence intervals for the parameter MLEs

Creating specific dictionary structure needed for the parameter 95% confidence interval plotting.

In [23]:
mu_lst_mx_ln = []
sigma_lst_mx_ln = []
omega_lst_mx_ln = []

mus_mx_ln = {}
sigmas_mx_ln = {}
omegas_mx_ln = {}

for group in sorted(group_vals, key=lambda x: order[x]):
    _smpls = bs_mle_samples_mx_ln[group]
    _mu_CI = np.percentile(_smpls[:, 0], [2.5, 97.5])
    _sigma_CI = np.percentile(_smpls[:, 1], [2.5, 97.5])
    _omega_CI = np.percentile(_smpls[:, 2], [2.5, 97.5])

    mu_mle, sigma_mle, omega_mle = ln_mles[group]

    _mu_dct = { 'label':group, 'conf_int':_mu_CI, 'estimate':mu_mle}
    _sigma_dct = { 'label':group, 'conf_int':_sigma_CI, 'estimate':sigma_mle}
    _omega_dct = { 'label':group, 'conf_int':_omega_CI, 'estimate':omega_mle}

    mu_lst_mx_ln.append(_mu_dct)
    sigma_lst_mx_ln.append(_sigma_dct)
    omega_lst_mx_ln.append(_omega_dct)

Plotting and saving 95% confidence interval plot for the mu parameter.

In [24]:
p_pub = plot_conf_int_pub(mu_lst_mx_ln, "mu", palette_CI)
bokeh.io.export_svgs(p_pub, filename=os.path.join("..", "Output", "nac_pole_mu_conf_int.svg"))

bokeh.io.save(
    p_pub,
    filename=os.path.join("..", "Output", "nac_pole_mu_conf_int.html"),
    title='Bokeh plot',
    resources=bokeh.resources.CDN,                          
);

bokeh.io.show(plot_conf_int_notebook(mu_lst_mx_ln, "mu", palette_CI))

Plotting and saving 95% confidence interval plot for the sigma parameter.

In [25]:
p_pub = plot_conf_int_pub(sigma_lst_mx_ln, "sigma", palette_CI)
bokeh.io.export_svgs(p_pub, filename=os.path.join("..", "Output", "nac_pole_sigma_conf_int.svg"))

bokeh.io.save(
    p_pub,
    filename=os.path.join("..", "Output", "nac_pole_sigma_conf_int.html"),
    title='Bokeh plot',
    resources=bokeh.resources.CDN,                          
);

bokeh.io.show(plot_conf_int_notebook(sigma_lst_mx_ln, "sigma", palette_CI))

Plotting and saving 95% confidence interval plot for the omega parameter.

In [26]:
p_pub = plot_conf_int_pub(omega_lst_mx_ln, "omega", palette_CI)
bokeh.io.export_svgs(p_pub, filename=os.path.join("..", "Output", "nac_pole_omega_conf_int.svg"))

bokeh.io.save(
    p_pub,
    filename=os.path.join("..", "Output", "nac_pole_omega_conf_int.html"),
    title='Bokeh plot',
    resources=bokeh.resources.CDN,                          
);

bokeh.io.show(plot_conf_int_notebook(omega_lst_mx_ln, "omega", palette_CI))

## Null hypothesis significance testing

For null hypothesis significance testing we decided to take the median across trials for every mouse to preserve 1 mouse = 1 data point data structure.

In [27]:
pole_df_long_median = pole_df_long.groupby(['Genotype', 'Treatment', 'ID'])['Time'].median().reset_index()

group_vals = {}

effect1_lst = pole_df_long_median[effect_1].unique()
effect2_lst = pole_df_long_median[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = pole_df_long_median.loc[(pole_df_long_median[effect_1] == i) & (pole_df_long_median[effect_2] == j), 'Time'].values
        if len(n) != 0:
            group_vals[name] = n

Kruskall-Wallis analysis of the groups:

In [28]:
p_val_ks = st.kruskal(group_vals['WT_V'], group_vals['ASO_V'], group_vals['WT_NAC'], group_vals['ASO_NAC'])
p_val_ks

KruskalResult(statistic=np.float64(16.44730742087835), pvalue=np.float64(0.0009179818331268663))

Post-hoc Conover pairwise comparison:

In [29]:
p_vals = sp.posthoc_conover([group_vals['WT_V'], group_vals['ASO_V'], group_vals['WT_NAC'], group_vals['ASO_NAC']], 
                            p_adjust='fdr_bh')
p_vals = p_vals.rename(columns={1:'WT_V', 2:'ASO_V', 3:'WT_NAC', 4:'ASO_NAC'}, 
                       index={1:'WT_V', 2:'ASO_V', 3:'WT_NAC', 4:'ASO_NAC'})
p_vals['KruskalWallis p-val'] = p_val_ks.pvalue
p_vals['KruskalWallis statistic'] = p_val_ks.statistic
p_vals.to_csv(os.path.join("..", "Output", "nac_pole_KW_conover.csv"))
p_vals

,WT_V,ASO_V,WT_NAC,ASO_NAC,KruskalWallis p-val,KruskalWallis statistic
WT_V,1.000000,0.001921,0.730527,0.017134,0.000918,16.447307
ASO_V,0.001921,1.000000,0.000904,0.280947,0.000918,16.447307
WT_NAC,0.730527,0.000904,1.000000,0.006836,0.000918,16.447307
ASO_NAC,0.017134,0.280947,0.006836,1.000000,0.000918,16.447307


In [30]:
%load_ext watermark
%watermark -v -p numpy,pandas,numba,bokeh,bebi103,jupyterlab

Python implementation: CPython
Python version       : 3.13.2
IPython version      : 9.0.2

numpy     : 2.1.3
pandas    : 2.2.3
numba     : 0.61.0
bokeh     : 3.7.0
bebi103   : 0.1.26
jupyterlab: 4.3.6

